In [1]:
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages')

import sklearn
import numpy as np
import csv

limit = 10e6
monomer_id_key = {}
monomer_id = 0
monomer_counts = np.zeros(int(10e5))
count = 0
target_dict = {}

db_name1 = 'purchase_target_10000.tsv'
db_name2 = 'BindingDB_All.tsv'
with open(db_name2) as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        if count>limit:
            break
        if count!=0:
            if len(line)>=9:
                target_name = line[6]
                current_monomer_id = line[4]
                Ki_value = line[8]
                if count%10000==0:
                    print(target_name)
                if Ki_value != '':
                    Ki_value = Ki_value.strip('<')
                    Ki_value = Ki_value.strip('>')
                    
                    if current_monomer_id not in monomer_id_key:
                        monomer_id_key[current_monomer_id] = monomer_id
                        monomer_id += 1
                        monomer_id_new = monomer_id
                    else:
                        monomer_id_new = monomer_id_key[current_monomer_id]
                    monomer_counts[monomer_id_new] += 1
                    if target_name in target_dict:
                        target_dict[target_name].append([monomer_id_new,float(Ki_value) ])
                    else:
                        target_dict[target_name] = []
                        target_dict[target_name].append([monomer_id_new,float(Ki_value) ])
        count += 1

Platelet-derived growth factor receptor
Matrix metalloproteinase-2 (MMP-2)
Aldose reductase (AR)
Asparaginyl Endopeptidase (AE)
Liver X receptor beta (LXRB)
Thrombin
fMet-Leu-Phe receptor 1/2
Glucocerebrosidase (Acid beta-Glucosidase)
glutathione S-transferase
POsterior Segregation family member (pos-1)
streptokinase A precursor
Hsf1 protein
Glycogen Synthase Kinase-3, beta
Intestinal alkaline phosphatase
Orexin receptor 1
Heat Shock 70kDa Protein 1
Voltage-gated potassium channel subunit Kv7.2
Histamine H1 receptor
DRD2
Acetylcholine Binding protein
Fructose-bisphosphate aldolase
Deoxyuridine 5&#39;-triphosphate nucleotidohydrolase (dUTPase)
Glutaminase 1
Liver X receptor beta (LXRB)
Prokineticin receptor 2 (PRK2)
Metabotropic Glutamate Receptor 5 (mGluR5)
Melanocortin receptor 4
Coagulation factor XI
Bromodomain-containing protein 3
Cathepsin A (CTSA)
Casein kinase II alpha/beta
P2X purinoceptor 7 (P2X7R)
AKT
Kappa Opioid Receptor
Histone deacetylase 3 (HDAC3)
Orexin receptor 2
Tyros

In [2]:
## We don't want too sparse a matrix, so we only use monomers that have 
## greater than n_data points
n_data = 6
n_removals = 0
ii = 0
monomer_mapping = np.arange(monomer_id+1)
monomer_counts_copy = np.copy(monomer_counts[0:monomer_id+1])
while ii<len(monomer_mapping):
    if monomer_counts_copy[ii]<n_data:
        #monomer_mapping[ii:-1] = monomer_mapping[ii:-1]-1
        monomer_mapping = np.delete(monomer_mapping, ii)
        monomer_counts_copy = np.delete(monomer_counts_copy, ii)
        n_removals += 1
    else:
        ii += 1
    if n_removals %10000 == 0:
        print(n_removals)

0
10000
20000
30000
40000
50000
50000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [85]:
n_monomers_per_target = 30
n_targets = 0
for target in target_dict:
    length = len(target_dict[target])
    if length > n_monomers_per_target:
        n_targets += 1

data_matrix = np.zeros((n_targets, len(monomer_mapping)))
target_id = 0
target_reverse_mapping = {}
target_mapping = {}

for target in target_dict:
    length = len(target_dict[target])
    if length > n_monomers_per_target:
        if target not in target_reverse_mapping:
            target_reverse_mapping[target] = target_id
            current_target_id = target_id
            target_mapping[target_id] = target
            target_id += 1
        else:
            current_target_id = target_reverse_mapping[target]     
        for ii in range(length):
            current_monomer_id = target_dict[target][ii][0]
            if monomer_counts[current_monomer_id]>n_data:
                new_monomer_id = np.where(monomer_mapping ==current_monomer_id)[0][0]
                Ki_value = target_dict[target][ii][1]
                data_matrix[current_target_id, new_monomer_id] = 1/Ki_value

In [4]:
target_mapping

{0: 'Liver X receptor beta (LXRB)',
 1: 'DNA Gyrase',
 2: 'Phenylethanolamine N-methyltransferase',
 3: 'Insulin-like growth factor 1 receptor (IGF1R)',
 4: '5-hydroxytryptamine receptor 2B (5HT2B)',
 5: '5-hydroxytryptamine receptor 2A (5-HT2A)',
 6: 'Metabotropic glutamate receptor 3',
 7: 'HSP90 and hnRNPA2/B1',
 8: '5-HT3A Serotonin Receptor',
 9: 'Phosphoinositide 3-kinases beta (PI3K beta)',
 10: 'Voltage-gated calcium channel alpha2/delta subunit 1',
 11: 'Subtilisin',
 12: 'Solute carrier organic anion transporter family member 1B3 (OATP1B3)',
 13: 'Histone Deacetylase 10 (HDAC10)',
 14: 'GABA receptor alpha-4/beta-3/gamma-2 subunit',
 15: 'HRH3',
 16: 'X-linked inhibitor of apoptosis protein (XIAP)',
 17: 'Matriptase',
 18: 'Acetylcholinesterase',
 19: 'Antrax lethal toxin',
 20: 'TACR2',
 21: 'TACR1',
 22: 'Trypsin I',
 23: 'Carboxypeptidase B',
 24: 'Calcitonin gene-related peptide type 1 receptor/Receptor activity-modifying protein 1',
 25: 'Orexin receptor 1',
 26: 'GABA t

In [86]:
## Get the column index for the small molecule positive example
## We find that we can just split the matrix as is and get the rough percents we want
positive_examples = np.reshape(np.where(data_matrix[805,:]!=0), 547)
print((positive_examples<5000).sum())

388


In [87]:
data_matrix_new = np.delete(data_matrix, 805, 0)

In [88]:
"Split into X_train, X_dev, X_test"
X_train = data_matrix_new[:,0:5000]
X_dev = data_matrix_new[:,5001:6071]
X_test = data_matrix_new[:,6072:7139]

Y_train = data_matrix[805,0:5000]
Y_dev = data_matrix[805,5001:6071]
Y_test = data_matrix[805,6072:7139]

In [95]:
np.savetxt('X_train.csv',X_train, delimiter=',')
np.savetxt('X_dev.csv',X_dev, delimiter=',')
np.savetxt('X_test.csv',X_test, delimiter=',')
np.savetxt('Y_train.csv',Y_train, delimiter=',')
np.savetxt('Y_dev.csv',Y_dev, delimiter=',')
np.savetxt('Y_test.csv',Y_test, delimiter=',')
